In [1]:
import numpy as np
import networkx as nx
import pandas as pd

# Function to simulate one time step of ICM model
def icm_step(graph, state, p):
    new_infected = set()
    for node in graph:
        if state[node] == 'I':
            for neighbor in graph.neighbors(node):
                if state[neighbor] == 'S' and np.random.random() < p:
                    new_infected.add(neighbor)
    for node in new_infected:
        state[node] = 'I'
    return new_infected

# Function to simulate ICM model for a specified number of time steps and simulations
def icm_simulation(graph, seeds, p, num_steps, num_sims):
    state = {node: 'S' for node in graph}
    for seed in seeds:
        state[seed] = 'I'
    
    cumulative_infected = []
    for _ in range(num_sims):
        state_copy = state.copy()
        infected_counts = [len(seeds)]
        for _ in range(num_steps):
            newly_infected = icm_step(graph, state_copy, p)
            infected_counts.append(len(newly_infected) + infected_counts[-1])
        cumulative_infected.append(infected_counts[-1])
    
    return cumulative_infected

# Load your network data and create a graph
G = nx.read_edgelist("12-Node-Network.txt", nodetype=int)

# Define parameters
seed_nodes = [1, 2, 3]  # Example seed nodes
beta_values = [0.01, 0.02, 0.03]  # Example beta values
num_time_steps = 40
num_simulations = 100

# Dictionary to store cumulative infected counts for each beta value
results = {}

# Perform simulations for each beta value
for beta in beta_values:
    cumulative_infected = icm_simulation(G, seed_nodes, beta, num_time_steps, num_simulations)
    results[beta] = cumulative_infected

# Convert results to DataFrame and save to Excel
df = pd.DataFrame.from_dict(results)
df.to_excel("ICM_cumulative_infected_results_ICM.xlsx", index=False)
